# Iris CLASSIFIER
### Let's build a quantum variational algorithm to classify the Iris flowers

In [1]:
# First, we import the libaries
import sys
import random

sys.path.append('../')
import quforge.quforge as qf
from quforge.quforge import State as State

In [2]:
#Load the dataset
from sklearn import datasets

iris = datasets.load_iris()
data = iris['data']
labels = iris['target']

ids = random.sample(range(len(data)), len(data))
ids_train = ids[0:int(0.8*len(data))]
ids_test = ids[int(0.8*len(data)):len(data)]

In [3]:
#Build the model

class Circuit(qf.Module):
    def __init__(self, dim, wires):
        super(Circuit, self).__init__()

        self.dim = dim
        self.init = qf.H(dim=dim, index=range(wires))
        self.encoder = qf.RZ(dim=dim, index=range(wires))

        self.layers = qf.Sequential(
            qf.RX(dim=dim, j=0, k=1, index=range(wires)),
            qf.RX(dim=dim, j=1, k=2, index=range(wires)),
            qf.RX(dim=dim, j=0, k=2, index=range(wires)),
            qf.RY(dim=dim, j=0, k=1, index=range(wires)),
            qf.RY(dim=dim, j=1, k=2, index=range(wires)),
            qf.RY(dim=dim, j=0, k=2, index=range(wires)),
            qf.RZ(dim=dim, j=1, index=range(wires)),
            qf.RZ(dim=dim, j=2, index=range(wires)),
            qf.CNOT(dim=dim, wires=wires, index=[0,1]),
            qf.CNOT(dim=dim, wires=wires, index=[0,2]),
            qf.CNOT(dim=dim, wires=wires, index=[0,3]),
            qf.RX(dim=dim, j=0, k=1, index=range(wires)),
            qf.RX(dim=dim, j=1, k=2, index=range(wires)),
            qf.RX(dim=dim, j=0, k=2, index=range(wires)),
            qf.RY(dim=dim, j=0, k=1, index=range(wires)),
            qf.RY(dim=dim, j=1, k=2, index=range(wires)),
            qf.RY(dim=dim, j=0, k=2, index=range(wires)),
            qf.RZ(dim=dim, j=1, index=range(wires)),
            qf.RZ(dim=dim, j=2, index=range(wires)),
        )

    def forward(self, x):
        y = State('0-0-0-0', dim=self.dim, device=device)
        y = self.init(y)
        y = self.encoder(y, param=x)
        y = self.layers(y)

        return y

In [4]:
#Instantiate model and optimizer
dim = 3 #dimension of the qudit
device = 'cuda'

model = Circuit(dim=dim, wires=4).to(device)
optimizer = qf.optim.Adam(model.parameters(), lr=0.001)

In [5]:
#Prepare the target states
targets = [State('1-0-0', dim=dim, device=device).flatten(), State('0-1-0', dim=dim, device=device).flatten(), State('0-0-1', dim=dim, device=device).flatten()]
targets_arg = [qf.argmax(abs(targets[0])), qf.argmax(abs(targets[1])), qf.argmax(abs(targets[2]))]

In [7]:
#Optimize the model
for epoch in range(8):
    acc_train = 0.0
    acc_test = 0.0
    
    for k in ids_train:
        x = data[k]
        label = labels[k]
        output = model(x)
        
        _, m = qf.measure(output, index=[0, 1, 2], dim=dim)
        
        loss = qf.mean(abs(targets[label] - m))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        predict = qf.argmax(m)
        if predict == targets_arg[label]:
            acc_train += 1

    for k in ids_test:
        x = data[k]
        label = labels[k]
        output = model(x)
        
        _, m = qf.measure(output, index=[0, 1, 2], dim=dim)

        predict = qf.argmax(m)
        if predict == targets_arg[label]:
            acc_test += 1
    
    acc_train = acc_train/len(ids_train)
    acc_test = acc_test/len(ids_test)

    print(epoch, acc_train, acc_test)

0 0.6916666666666667 0.5
1 0.6916666666666667 0.5333333333333333


KeyboardInterrupt: 